In [ ]:
%load_ext nb_black

# [原文地址](https://www.toutiao.com/i6694196616334672387/)

In [3]:
import requests
from bs4 import BeautifulSoup

user_agent = "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.106 Safari/537.36"
referer = "http://www.xicidaili.com/"
headers = {"user-agent": user_agent, "referer": referer}
target = "http://www.xicidaili.com/nt/"
# 获取页面源码
r = requests.get(target, headers=headers)

In [4]:
# 解析页面源码
soup = BeautifulSoup(r.text, "lxml")
for tr in soup.find_all("tr")[1:]:
    tds = tr.find_all("td")
    proxy = tds[1].text + ":" + tds[2].text
    print(proxy)

222.143.158.152:80
42.228.3.158:8080
219.131.243.172:9797
122.51.207.244:8888
220.168.86.37:3128
183.63.188.250:8808
116.196.87.86:20183
14.153.53.30:3128
118.114.194.59:8118
163.125.71.152:8888
119.129.84.27:1080
59.38.62.44:808
125.123.18.144:9000
36.112.128.235:3128
218.249.69.214:1081
1.83.117.55:8118
112.253.11.103:8000
123.117.39.139:9000
183.136.177.77:3128
113.68.24.185:1080
121.69.46.177:9000
14.20.235.7:808
122.228.19.7:3389
163.125.18.59:8888
163.125.17.220:8888
111.206.118.106:8080
27.38.96.174:9797
121.41.122.40:8118
124.237.83.14:53281
117.62.56.96:8118
218.27.136.169:8085
113.65.8.112:9797
58.17.125.215:53281
61.145.182.27:53281
112.64.233.130:9991
115.195.84.31:8118
110.16.80.106:8080
111.164.20.86:8111
119.131.91.47:9797
58.251.234.176:9797
60.191.11.246:3128
163.125.69.232:8888
49.84.161.6:53281
223.245.45.90:65309
218.75.158.153:3128
116.196.85.166:3128
222.223.182.66:8000
218.2.226.42:80
61.153.251.150:22222
123.58.17.134:3128
120.76.117.20:3128
221.13.156.158:55443

In [7]:
# {'http':'61.135.217.7:80'},
# {'http':'182.88.161.204:8123'}

# 解析页面源码
soup = BeautifulSoup(r.text, "lxml")
for tr in soup.find_all("tr")[1:]:
    tds = tr.find_all("td")
    proxy = "{'"+tds[5].text+"':'"+tds[1].text + ":" + tds[2].text+"'},"
    print(proxy)

{'HTTP':'222.143.158.152:80'},
{'HTTPS':'42.228.3.158:8080'},
{'HTTPS':'219.131.243.172:9797'},
{'HTTP':'122.51.207.244:8888'},
{'HTTP':'220.168.86.37:3128'},
{'HTTP':'183.63.188.250:8808'},
{'HTTP':'116.196.87.86:20183'},
{'HTTP':'14.153.53.30:3128'},
{'HTTP':'118.114.194.59:8118'},
{'HTTP':'163.125.71.152:8888'},
{'HTTP':'119.129.84.27:1080'},
{'HTTP':'59.38.62.44:808'},
{'HTTP':'125.123.18.144:9000'},
{'HTTPS':'36.112.128.235:3128'},
{'HTTP':'218.249.69.214:1081'},
{'HTTP':'1.83.117.55:8118'},
{'HTTP':'112.253.11.103:8000'},
{'HTTP':'123.117.39.139:9000'},
{'HTTP':'183.136.177.77:3128'},
{'HTTP':'113.68.24.185:1080'},
{'HTTP':'121.69.46.177:9000'},
{'HTTPS':'14.20.235.7:808'},
{'HTTP':'122.228.19.7:3389'},
{'HTTPS':'163.125.18.59:8888'},
{'HTTP':'163.125.17.220:8888'},
{'HTTPS':'111.206.118.106:8080'},
{'HTTP':'27.38.96.174:9797'},
{'HTTP':'121.41.122.40:8118'},
{'HTTPS':'124.237.83.14:53281'},
{'HTTP':'117.62.56.96:8118'},
{'HTTP':'218.27.136.169:8085'},
{'HTTP':'113.65.8.112:9797'

In [ ]:
import redis

HOST = "127.0.0.1"  # redis所在主机IP
PORT = 6379  # redis服务监听的端口
PASSWORD = ""  # 连接redis的密码
DB = 0  # IP存储的DB编号


def get_connection():
    r = redis.Redis(host=HOST, port=PORT, password=PASSWORD, db=DB)
    return r

In [ ]:
import requests
from bs4 import BeautifulSoup

# 获取redis连接
try:
    conn = get_connection()
except Exception:
    print("Error while connecting to redis")

user_agent = "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.106 Safari/537.36"
referer = "http://www.xicidaili.com/"
headers = {"user-agent": user_agent, "referer": referer}
target = "http://www.xicidaili.com/nt/"
# 获取页面源码
r = requests.get(target, headers=headers)
# 解析页面源码
soup = BeautifulSoup(r.text, "lxml")
for tr in soup.find_all("tr")[1:]:
    tds = tr.find_all("td")
    proxy = tds[1].text + ":" + tds[2].text
    conn.sadd("ip_set", proxy)
    print("%s added to ip set" % proxy)

In [ ]:
# 判断IP是否能访问目标网站
def is_valid(url, ip):
    proxy = {"http": "http://%s" % ip}
    user_agent = "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.106 Safari/537.36"
    headers = {"user-agent": user_agent}
    try:
        print(proxy)
        print(url)
        r = requests.get(url, headers=headers, proxies=proxy, timeout=6)
        print(r)
        return True
    except Exception:
        return False


my_proxy, proxies, ip_set = None, None, "amazon_ips"
conn = get_connection()
target = "https://www.amazon.com/"

while not is_valid(target, my_proxy):
    if my_proxy:
        conn.srem(ip_set, my_proxy)  # 删除无效的代理IP
    if proxies:
        my_proxy = proxies.pop()
    else:
        proxies = conn.srandmember(ip_set, 5)  # 从redis中随机抽5个代理ip
        my_proxy = proxies.pop()
        print("valid proxy %s" % my_proxy)